# Pandasの使い方

Pandas は、表形式データ（行と列）を扱うためのライブラリです。
CSV の読み込み、欠損値処理、集計、結合といった分析前処理の中心になります。

このノートでは、実務で頻出する `選択 → 加工 → 集計 → 結合` の流れを一通り体験します。

In [ ]:
import numpy as np
import pandas as pd


まずは DataFrame を手で作って、行・列の見方を掴みます。

In [ ]:
df = pd.DataFrame({
    "student": ["A", "B", "C", "D", "E"],
    "grade": [1, 1, 2, 2, 2],
    "math": [72, 88, 91, 67, 79],
    "english": [75, 90, 85, 70, 82],
    "club": ["AI", "Art", "AI", "Sports", "AI"],
})

df


列選択と条件抽出は、Pandas の最も基本的な操作です。

In [ ]:
print(df[["student", "math"]])
print("---")
print(df[df["math"] >= 80])


新しい列を追加するときは、計算式を明示的に書きます。
ここでは合計点と判定ラベルを作ります。

`np.where(condition, true_value, false_value)` は、
各行ごとに条件を判定して値を返すベクトル化された分岐です。

In [ ]:
df["total"] = df["math"] + df["english"]
df["pass"] = np.where(df["total"] >= 160, "pass", "retry")

df


`groupby` は「同じカテゴリごとに集計する」操作です。
クラブ別の平均点を見ると、分布の傾向をつかめます。

`agg` の `students=("student", "count")` は、
「出力列名を students にして、student 列へ count 集計をかける」という意味です。
`as_index=False` を指定しているので、集計キーの `club` を列として残したまま結果を扱えます。

In [ ]:
club_summary = (
    df.groupby("club", as_index=False)
      .agg(
          students=("student", "count"),
          math_mean=("math", "mean"),
          english_mean=("english", "mean"),
      )
)

club_summary


欠損値（NaN）は実データで必ず出ます。
埋め方を決める前に、どこが欠損しているかを確認するのが先です。

In [ ]:
df_missing = df.copy()
df_missing.loc[2, "english"] = np.nan

print(df_missing)
print("--- 欠損数 ---")
print(df_missing.isna().sum())


今回はシンプルに列平均で埋めます。実務では、意味を踏まえて中央値やモデル補完を選びます。

In [ ]:
filled = df_missing.copy()
filled["english"] = filled["english"].fillna(filled["english"].mean())
filled


表の結合（merge）は分析の中核です。キー列を揃えてから結合します。
ここでは `how="left"` を使い、左側（成績表）の行を必ず残しながら、
一致する `student` があればプロフィール列を付与します。一致しない場合は NaN になります。

In [ ]:
profile = pd.DataFrame({
    "student": ["A", "B", "C", "D", "E"],
    "city": ["Tokyo", "Osaka", "Nagoya", "Fukuoka", "Tokyo"],
    "hours": [6.5, 4.0, 7.0, 3.5, 5.0],
})

merged = pd.merge(df, profile, on="student", how="left")
merged


縦持ち・横持ちの変換もよく使います。`melt` で縦持ちにすると可視化しやすくなります。

- `id_vars`: そのまま残す識別列（ここでは student, club）
- `value_vars`: 積み上げて1列にまとめる列（ここでは math, english）
- `var_name`, `value_name`: 変換後の列名（ここでは subject, score）

In [ ]:
long_df = merged.melt(
    id_vars=["student", "club"],
    value_vars=["math", "english"],
    var_name="subject",
    value_name="score"
)

long_df.head(10)


日付列を持つデータでは、`datetime` に変換してから月単位などで集計します。

In [ ]:
sales = pd.DataFrame({
    "date": ["2026-01-02", "2026-01-14", "2026-02-03", "2026-02-10", "2026-03-01"],
    "amount": [120, 95, 140, 110, 180]
})

sales["date"] = pd.to_datetime(sales["date"])
sales["month"] = sales["date"].dt.to_period("M")
monthly = sales.groupby("month", as_index=False)["amount"].sum()
monthly


Pandas では、何をキーに行を合わせるか、どの列を残すかを明確にすると処理が安定します。
前処理はモデル以前の品質を決めるので、読みやすい集計コードを意識して書くのが重要です。